In [44]:
import rpy2.robjects as robjects
from rpy2.robjects import numpy2ri
import rpy2
from pprint import pprint
import re
import numpy as np
import pickle
from skbio.alignment import StripedSmithWaterman

In [2]:
path = "../../david_eccles_bc_ideas/"
SRATE = 5000

In [3]:
robjects.r['load']("../../david_eccles_bc_ideas/mtDNA_mappable_called_with_rawSignal.Rdata")

<StrVector - Python:0x7f82d95d5cc8 / R:0x3794458>
['nanopore.bundled']

In [4]:
data = numpy2ri.ri2py(robjects.r['nanopore.bundled'])

In [5]:
files = !ls ../../david_eccles_bc_ideas/processed/ | grep fast5

In [6]:
n = 0
file = files[n]

In [7]:
result = re.search(r'_ch(\d+)_file(\d+)_', file)
file_id = result.group(2)
channel_id = result.group(1)
print(channel_id, file_id)

132 10


In [42]:
ref = data[0][0]

only look at template strand for the beginning

In [8]:
called_seq = !poretools fasta {path+"processed/" + file} | grep template -A1 | grep -v ">"
called_seq = called_seq[0]

In [9]:
events = !poretools events {path+"processed/" + file} | grep template

file	strand	mean	start	stdv	length	model_state	model_level	move	p_model_state	mp_model_state	p_mp_model_state	p_A	p_C	p_G	p_T	raw_index

In [10]:
events = [line.split("\t") for line in events]

In [11]:
start = round(float(events[0][3]) * 5000)
end = round(float(events[-1][3]) * 5000)
print (start, end)

14263383 15169026


0 = reference
1 = called
3 = channel
8 = metrichor results (events)
9 = "mux"
10 = signal range 
11 = signal

In [13]:
types = list(data[2].names)

In [14]:
## channel coords from rdata
coords = []
for i in range(0, len(data[10]), 2):
    coords.append((round(data[10][i]), round(data[10][i+1])))

all_raw = np.empty(round(max(data[10])))
all_raw.fill(-1)

convert the chunked signal into one accessible array

In [36]:
# for i, start_end in enumerate(coords):
#     start, end = start_end
#     print(i, start)
#     for k, sig in enumerate(data[11][i]):
#         all_raw[start + k] = sig
all_raw = pickle.load(open("david_raw.pickle", 'rb'))

In [16]:
!makeblastdb -in david_mmu_ref.fa -parse_seqids -dbtype nucl



Building a new DB, current time: 07/13/2015 13:02:51
New DB name:   /home/sturm/repos/uni/nanopore/own/notebooks/david_mmu_ref.fa
New DB title:  david_mmu_ref.fa
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/sturm/repos/uni/nanopore/own/notebooks/david_mmu_ref.fa
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.00302196 seconds.


In [17]:
fasta = '>channel_{0}_file_{1}\n{2}'.format(channel_id, file_id, called_seq)
!echo "{fasta}" | blastn -db "david_mmu_ref.fa"

BLASTN 2.2.31+


Reference: Zheng Zhang, Scott Schwartz, Lukas Wagner, and Webb
Miller (2000), "A greedy algorithm for aligning DNA sequences", J
Comput Biol 2000; 7(1-2):203-14.



Database: david_mmu_ref.fa
           1 sequences; 16,299 total letters



Query= channel_132_file_10

Length=7058
                                                                      Score     E
Sequences producing significant alignments:                          (Bits)  Value

mmus_mtDNA                                                            353     4e-99


>mmus_mtDNA 
Length=16299

 Score = 353 bits (191),  Expect = 4e-99
 Identities = 795/1058 (75%), Gaps = 156/1058 (15%)
 Strand=Plus/Plus

Query  6093  CATGCTTTTGTTATAGATTTTCTTCGCTCG----GCC-GTAATAATTGGAGGCGTTGGAA  6147
             ||||||||||||||| |||||||||  | |     ||  |||||||||||||| ||||||
Sbjct  5508  CATGCTTTTGTTATA-ATTTTCTTC-ATAGTAATACCAATAATAATTGGAGGCTTTGGAA  5565

Query  6148  AAT-A-GTGTCCCACGGGTCAATC-GA-ACGC--A-CT-CCATTCCCACCGAT-AGTAAT  61

In [45]:
align_ref = StripedSmithWaterman(ref)

In [60]:
align_ref(called_seq)

{
    'optimal_alignment_score': 2380,
    'suboptimal_alignment_score': 0,
    'query_begin': 22,
    'query_end': 6535,
    'target_begin': 1053,
    'target_end_optimal': 7022,
    'target_end_suboptimal': 0,
    'cigar': '8M1I22M2I6M3I5M2I5M1D7M3I8M1D11M2I8M1I3M1I5M1D19M1I14M1I7M2I13M1I5M1I3M4I19M3I2M7I16M4I17M1I21M2D14M2I14M5D19M2I20M4I7M1D2M1D6M2I12M1I10M2I13M2I4M1I4M1D5M2I2M2I14M1I21M2I20M3I44M1D4M1I7M1I1M1I6M2I1M2I13M3I11M1I16M3D8M1I8M2D5M3I4M4I4M3D6M1I2M2D8M2I9M8I3M1I7M1I5M8I4M2I4M2I14M2I10M1I3M2I5M1D4M3I1M3I7M2I8M1I1M1I6M4I10M1I2M1I18M2I2M2I4M1I2M3I6M1D1M1D4M1I12M1I15M2D2M1I4M2I1M1I11M3D5M2D4M3D14M7D4M7D1M5D2M13D5M2D6M8D4M3D5M1I3M1D6M1D1M12D2M2D2M1D5M5D9M21D8M3D2M12D18M2I7M1D22M2I15M4I4M1I1M2I17M1I10M2I1M1I3M1I24M5I2M1I14M2I3M5I2M1D22M1I3M2I7M1I6M1I2M1D4M1I12M1I7M2I1M1I9M2I13M1I1M1I23M3I9M2I19M1I4M1I4M1I9M1I2M1I7M1I9M1I16M1I7M1I4M1I8M5I18M1D6M1I3M2D3M1I6M4D25M2I7M2I21M1I13M1D2M2I11M5I11M2I11M5D9M1I6M1I5M3I26M4I11M1D5M1I5M2I30M3I6M1I8M1I9M3I5M1I2M1I13M1I2M1I14M1I6M1I4M2I8M1I2M

In [61]:
len(called_seq)

7058